In [50]:
from tensorflow.keras.models import load_model
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from joblib import load
import numpy as np
import geopandas as gpd

In [84]:
# Load Model
model = load_model('../static/data/nnn_model_1.h5')


In [39]:
# Prep Input
# Predict Inputs

input_lng = -111.88080951152304
input_lat = 41.08032456289652
input_date = '2020-11-18'
predict_input = pd.DataFrame({'longitude':input_lng,'latitude':input_lat,'date':input_date}, index = [0])

In [40]:
# Convert strings to datetime objects
begin_date_obj = datetime.strptime(input_date, "%Y-%m-%d")
end_date_obj = begin_date_obj + timedelta(days=1)

# Convert back to string
end_date = end_date_obj.strftime("%Y-%m-%d")

In [41]:
# Set up sqlite
connection = sqlite3.connect('../static/data/sensors_readings_2016_present.db')

# Assemble Query
sql_query = """
SELECT sensor_id, pm2
FROM sensors_readings
WHERE date(date) BETWEEN ? AND ?
"""

# Execute the query
df = pd.read_sql_query(sql_query, connection, params=(input_date, end_date))

connection.close()
df

,sensor_id,pm2
0,18321,2.6435
1,6356,0.2815
2,18259,1.5640
3,6298,1.2870
4,5742,2.0365
...,...,...
222,62211,0.9895
223,31013,2.6375
224,18375,1.1360
225,6050,1.2960


In [42]:
# merge with sensor cateogires
df_cat = pd.read_csv('../static/data/sensor_categories.csv')
df = df.merge(df_cat, on = 'sensor_id')
df.head()

,sensor_id,pm2,category,county
0,18321,2.6435,green,Salt Lake County
1,6356,0.2815,orange,Salt Lake County
2,18259,1.5640,green,Salt Lake County
3,6298,1.2870,green,Davis County
4,5742,2.0365,blue,Salt Lake County


In [43]:
df = df[['county','pm2']]

In [44]:
df = df.groupby('county').mean()

In [45]:
df.reset_index(inplace = True)

In [67]:
predict_input.dtypes

longitude    float64
latitude     float64
date          object
dtype: object

In [68]:
url = '../static/data/centroids_merge.csv'
predict_input = predict_input.merge(pd.read_csv(url), on = ['latitude','longitude'])


In [69]:
predict_input = predict_input

,longitude,latitude,date,Unnamed: 0,county,category
0,-111.88081,41.080325,2020-11-18,0,Davis County,blue


In [70]:
predict_input['month'] = begin_date_obj.month

# Print the column names
predict_input['avg_pm2'] = df.loc[df.county == list(true_column)[0].split('_')[1]].pm2.values[0]


In [71]:
predict_input

,longitude,latitude,date,Unnamed: 0,county,category,month,avg_pm2
0,-111.88081,41.080325,2020-11-18,0,Davis County,blue,11,1.268242


In [72]:
category_map = {'red':1,'orange':2,'green':3,'blue':4}
county_map = {'Salt Lake County':1,'Weber County':2,'Davis County':3}
df = predict_input
df['county_encoded'] = df['county'].map(county_map)
df['category_encoded'] = df['category'].map(category_map)

In [114]:
#temp at avg_pm2
df['avg_pm10'] = 30.5
df['avg_pm2'] = 17.5

/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_83520/1591629937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_pm10'] = 30.5
/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_83520/1591629937.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_pm2'] = 17.5


In [121]:
#f = df.drop(['Unnamed: 0','county','category'],axis=1)
df.head(1)
df.county_encoded = 1

/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_83520/1414550929.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.county_encoded = 1


In [122]:
df = df[['latitude','longitude', 'avg_pm10','avg_pm2','county_encoded','category_encoded','month']]

In [128]:
# Load the preprocessor
preprocessor = load('../static/data/preprocessor.joblib')

# Now you can transform new data using the loaded preprocessor
X_new_transformed = preprocessor.transform(df)


In [129]:
X_new_transformed

array([[ 1.23212883,  0.4032256 , -0.71148417, -0.51249774,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ]])

In [130]:
prediction = model.predict(X_new_transformed)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [131]:
prediction

array([[2.7345862]], dtype=float32)

In [127]:
np.expm1(prediction)

array([[-0.36650077]], dtype=float32)

In [102]:
np.round(np.expm1(prediction)[0])


array([-0.], dtype=float32)

In [ ]:

        pm2 = np.round(np.expm1(prediction)[0])
        
        # Ensure the input_value is shaped correctly
        input_value = np.array([[pm2], [pm10]])  # Shape (1, 2)
        
        # Predict and round the prediction
        prediction = model.predict(input_value)
        prediction = np.round(prediction).flatten().tolist()  # Flatten and convert to list
        
        # Return the prediction as a JSON response
        return jsonify(prediction)
    except Exception as e:
        return jsonify({"error": str(e)}), 500